In [ ]:
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
from __future__ import print_function
import time as t
from googleapiclient import discovery
from httplib2 import Http
from oauth2client import file, client, tools

import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
dataceunsp = pd.read_csv('data_ceunsp_F2.csv',sep=',')

In [ ]:
SHEETS_FILE_ID = '1g36yCFTmeA2Q5soV48_01Agebm7I-vFxgsWEeenRKf8'

# authorization constants 
CLIENT_ID_FILE = 'credentials.json'
TOKEN_STORE_FILE = 'token.json'
SCOPES = (  # iterable or space-delimited string
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/documents',
    'https://www.googleapis.com/auth/spreadsheets.readonly',
)
# application constants
SOURCES = ('text', 'sheets')
SOURCE = 'sheets' # Choose one of the data SOURCES

def get_http_client():
    """Uses project credentials in CLIENT_ID_FILE along with requested OAuth2
        scopes for authorization, and caches API tokens in TOKEN_STORE_FILE.
    """
    store = file.Storage(TOKEN_STORE_FILE)
    creds = store.get()
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_ID_FILE, SCOPES)
        creds = tools.run_flow(flow, store)
    return creds.authorize(Http())

# service endpoints to Google APIs
HTTP = get_http_client()
DRIVE = discovery.build('drive', 'v3', http=HTTP)
DOCS = discovery.build('docs', 'v1', http=HTTP)
SHEETS = discovery.build('sheets', 'v4', http=HTTP)

def get_data(source):
    """Gets mail merge data from chosen data source.
    """
    if source not in {'sheets', 'text'}:
        raise ValueError('ERROR: unsupported source %r; choose from %r' % (
            source, SOURCES))
    return SAFE_DISPATCH[source]()

def set_data(source,NOMES,service=SHEETS):
    """Set data from chosen data source.
    """
    if source not in {'sheets', 'text'}:
        raise ValueError('ERROR: unsupported source %r; choose from %r' % (
            source, SOURCES))
    RANGE = 'G2:G'+str(len(NOMES)+1)
    values = NOMES
    body = {
        'values': values
    }
    request = service.spreadsheets().values().update(spreadsheetId=SHEETS_FILE_ID, range=RANGE, valueInputOption='RAW', body= body)
    response = request.execute()    
    return SAFE_DISPATCH[source]()

def _get_sheets_data(service=SHEETS):
    """(private) Returns data from Google Sheets source. It gets all rows of
        'Sheet1' (the default Sheet in a new spreadsheet), but drops the first
        (header) row. Use any desired data range (in standard A1 notation).
    """
    return service.spreadsheets().values().get(spreadsheetId=SHEETS_FILE_ID,
            range='Sheet1').execute().get('values')[1:] # skip header row

SAFE_DISPATCH = {k: globals().get('_get_%s_data' % k) for k in SOURCES}

In [ ]:
data = get_data(SOURCE) # get data from data source
NOME = []
RGM = []
CURSO = []
for i, row in enumerate(data):
    NOME.append(str(data[i][3])) 
    RGM.append((data[i][4]))
    CURSO.append(str(data[i][5]))    

In [ ]:
matriz = [NOME, RGM, CURSO]
matriz = np.array(matriz) 
df_forms = pd.DataFrame(matriz.T, columns='NOME RGM CURSO'.split())
df_forms['RGM']=df_forms['RGM'].astype(int)  

In [ ]:
dataceunsp = dataceunsp[['RGM','NOME']]

In [ ]:
aux1= pd.merge(df_forms,dataceunsp,how = 'left', on = ['RGM'],suffixes=('_aluno','_ceunsp'))

In [ ]:
aux2 = aux1[aux1['NOME_ceunsp'].isnull()].drop(columns='NOME_ceunsp').rename(columns={'NOME_aluno':'NOME'})
aux3 = pd.merge(aux2,dataceunsp,how = 'left', on = ['NOME'],suffixes=('_aluno','_ceunsp'))

In [ ]:
result = {}
NOMES = []
for nome_as1 in df_forms['NOME']:
    CURSO = df_forms[df_forms['NOME'] == nome_as1]['CURSO'].to_string()
    if 'ALUNO DE DP/ADAP' in CURSO:
        NOMES.append([nome_as1])      
    else:     
        for nome_ceunsp in dataceunsp['NOME']:
            nome_as1 = nome_as1.strip()
            result[nome_ceunsp]=similar(nome_as1,str(nome_ceunsp))
        sort_result = sorted(result.items(),key = lambda x: x[1],reverse = True)
        NOMES.append([sort_result[0][0]])
        if sort_result[0][1] < .5:
            sort_result[0] = ''
#        print (nome_as1,sort_result[0])    
        
set_data(SOURCE,NOMES)   